In [1]:
import numpy as np
import pandas as pd
import torch
import zipfile
import kaggle
import os
import math

from pathlib import Path

In [2]:
print('hello we building random forest algo')

hello we building random forest algo


In [3]:
# Download dataset to use locally

path = Path('titanic')
extract_path = Path('data/titanic')
if not path.exists():
    kaggle.api.competition_download_cli(str(path))
    zipfile.ZipFile(f'{path}.zip').extractall(extract_path)
    
assert os.path.isdir(extract_path) == True, f"Directory '{path}' should be created at this point"
assert len(os.listdir(extract_path)) > 0, f"Directory '{path}' is empty somehow, we expect files here"

titanic.zip: Skipping, found more recently modified local copy (use --force to force download)


In [4]:
df = pd.read_csv(f'{extract_path}/train.csv')
# df.head()

In [5]:
# Handle missing values in the dataset

modes = df.mode().iloc[0]
df.fillna(modes, inplace=True)
# df.isna().sum()
# df.head()

In [88]:
features = ['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked']
# sibsp	-- # of siblings / spouses aboard the Titanic
# parch -- # of parents / children aboard the Titanic
# embarked -- Port of Embarkation C = Cherbourg, Q = Queenstown, S = Southampton

In [104]:
train_index = int(np.floor(0.9 * len(df)))

X_tr = df[features][:train_index]
Y_tr = df['Survived'][:train_index].values
X_test = df[features][train_index:]
Y_test = df['Survived'][train_index:].values

In [105]:
X_tr

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,3,male,22.0,1,0,7.2500,S
1,1,female,38.0,1,0,71.2833,C
2,3,female,26.0,0,0,7.9250,S
3,1,female,35.0,1,0,53.1000,S
4,3,male,35.0,0,0,8.0500,S
...,...,...,...,...,...,...,...
796,1,female,49.0,0,0,25.9292,S
797,3,female,31.0,0,0,8.6833,S
798,3,male,30.0,0,0,7.2292,C
799,3,female,30.0,1,1,24.1500,S


In [109]:
# df to tenrsors
if isinstance(X_tr, pd.DataFrame):
    X_tr = pd.get_dummies(X_tr, columns=['Sex', 'Embarked'])
    i_to_column = {i: c for i,c in enumerate(X_tr.columns)}
    X_tr = torch.tensor(X_tr.astype(np.float32).values, dtype=torch.float32)
    Y_tr = torch.tensor(Y_tr, dtype=torch.float32)

In [110]:
print(i_to_column)

{0: 'Pclass', 1: 'Age', 2: 'SibSp', 3: 'Parch', 4: 'Fare', 5: 'Sex_female', 6: 'Sex_male', 7: 'Embarked_C', 8: 'Embarked_Q', 9: 'Embarked_S'}


In [111]:
class Sampler:
    def bootstrap(self, x, y):
        n = len(x)
        indices = np.random.choice(n, size=n, replace=True)
        return x[indices], y[indices]

In [112]:
sampler = Sampler()

In [113]:
X_tr, Y_tr = sampler.bootstrap(X_tr, Y_tr)

In [114]:
X_tr[0]

tensor([ 3.0000, 30.0000,  0.0000,  0.0000,  8.0500,  0.0000,  1.0000,  0.0000,
         0.0000,  1.0000])

In [132]:
def entropy(p):
    '''
    How uncertain our split is. If values are 50/50 -- very uncertain, 1. 
    If nearly all values are the same -- certainty is high, low entr
    '''
    if p == 0:
        return 0
    elif p == 1:
        return 0
    else:
        return - (p * np.log2(p) + (1 - p) * np.log2(1-p))

assert entropy(.5) == 1
assert entropy(.99) < 0.1

In [168]:
def information_gain(left_child, right_child):
    '''
    Calculates how much uncertainty is reduced by doing the split. Higher gain = better split.
    Assumes binary classification with labels 0 and 1.
    '''

    # Combine both child sets to get the parent
    parent = torch.cat((left_child, right_child), dim=0)

    # Safeguard against division by zero
    def proportion_ones(t):
        return torch.sum(t == 1).item() / t.numel() if t.numel() > 0 else 0.0

    # Proportions of class 1
    p_parent = proportion_ones(parent)
    p_left = proportion_ones(left_child)
    p_right = proportion_ones(right_child)

    # Entropies
    IG_p = entropy(p_parent)
    IG_l = entropy(p_left)
    IG_r = entropy(p_right)

    # Weights
    l_weight = left_child.numel() / parent.numel()
    r_weight = right_child.numel() / parent.numel()

    return IG_p - l_weight * IG_l - r_weight * IG_r

In [133]:
def information_gain(left_child, right_child):
    '''
    It tells us how much "uncertainty" is reduced by doing the split. Higher gain = better split.
    '''
    
    # original set before split
    parent = left_child + right_child 

    # propostions in the original set, on the left and right splits
    p_parent = parent.count(1) / len(parent) if len(parent) > 0 else 0
    p_left = left_child.count(1) / len(left_child) if len(left_child) > 0 else 0
    p_right = right_child.count(1) / len(right_child) if len(right_child) > 0 else 0
    
    IG_p = entropy(p_parent)
    IG_l = entropy(p_left)
    IG_r = entropy(p_right)
    
    l_weight = len(left_child) / len(parent)
    r_weight = len(right_child) / len(parent)
    
    return IG_p - l_weight * IG_l - r_weight * IG_r

In [182]:
class DecisionTree:
    def __init__(self, max_depth=5, min_samples_split=2, feature_subsample=None):
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.feature_subsample = feature_subsample

    def straightforward(self, x, y, depth):
        if depth == self.max_depth:
            return 0

        best_split = None
        ig_max = -9999
        
        for feature in self.feature_subsample:
            possible_splits = x[:, feature].unique()

            for split in possible_splits:
                left_mask = x[:, feature] <= split
                right_mask = x[:, feature] > split
                
                left = x[left_mask]
                right = x[right_mask]
                
                ig = information_gain(left, right)
                
                if ig > ig_max:
                    ig_max = ig
                    best_split = split

        straightforward(self, 


            
        
    
    def fit(self, x, y):
        # Entry point — builds the tree recursively
        return 0

    def _best_split(self, x, y, features):
        # Try all features and thresholds, return best info gain split
        return 0

    def _build_tree(self, x, y, depth):
        # Recursively build tree nodes
        return 0

    def predict(self, x):
        # Traverse the built tree for one sample
        return 0


In [179]:
tree = DecisionTree(feature_subsample=[0,1,2])

In [180]:
tree.straightforward(X_tr, Y_tr)

0.008278304605820352
0.004865368629391242
0.0
1.4150037967897156e-05
0.00014535822069361437
0.0002977953583942483
0.00043297558988819596
0.0003734745106582782
0.000381033891895477
0.0003886322906273554
0.00037654113281848645
0.0003755651842836105
0.0002799874750366582
0.0002129285627112676
0.00019332218245016186
0.00016451488526492142
0.00020197507263808312
0.00032002380566986677
0.0003061066212854957
0.0003827205216976326
0.00030263787192053027
0.0002467094525999691
0.00018920765354735192
6.451719405242695e-05
1.539125940386299e-05
2.0253958822213924e-05
1.7384675270015748e-05
0.0002214938608304795
0.00023028365692884245
0.00020385913271331324
0.00026543541735940535
0.0003730283503298182
0.000555318365714097
0.0005700126067113009
0.0006884439508646478
0.00098150011896464
0.001002373670004164
0.0010548840669642456
0.0013918279429539493
0.001405958234196536
0.0015081309094884254
0.0014415847949122984
0.001287846944596105
0.0011182393248730937
0.0010601880923733764
0.001116162351962935
0